In [18]:
# Author: Mohammed Alsoughayer
# Descritpion: PhysionTrainerBot implimentation

# Import necessary packages
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import tensorflow as tf
import json
import string
import random
import nltk
from nltk.stem import WordNetLemmatizer # It has the ability to lemmatize.
from keras import Sequential # Sequential groups a linear stack of layers into a tf.keras.Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation, Bidirectional,Embedding
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences

# Introduction 
Physical health is a common goal amongst all humans; whether we are injured or just simply aren’t as healthy as we want, having a well established exercise routine will go a long way in our physical and mental status. However, putting together that routine can be very cumbersome to some people and might halt their motivation to become healthier. Moreover, going to a physical therapist or getting a personal trainer to help put together a routine might be too expensive for some people. That being said, my goal is to build a web-app consisting of chatbot to converse with any user. The purpose of the chatbot is to serve as a free physical therapist/trainer. It would converse with the user to understand their physical situation and goal. Then It will recommend a training plan to aid them reach the physical health they desire. Disclaimer: The chatbot analysis for injured users shouldn’t be taken as a diagnosis, rather, it should serve as an educational tool to help the user in understanding the possible condition of their health and the exercises that could aid their physical health recovery.

# Current Version
As of right now, PhysioTrainerBot uses NLP techniques to identify the intent of the user from the following options: strength, stamina, or weight-loss. Then depending on their goal it will generate 7-day routine.

# Journy 
## NLP 
To start I implimented a general chatbot using a pretrained model to understand the workflow and pick up inspirations for my implimentation. Afterwards, I did a basic implimentation that only interprets specific promts. Then I tried to generalize the NLU and NLG process, but got too overwhelmed. Therefore, I improved on the basic implimentation to be able to understand intents for various prompts. 
## Injury Classification  
Most of time time developing this module was spent researching medical practices. After realizing the complexity of the diagnostic procedure, I tried implemnting a web-crawler that uses nlp techniques to build a health profile for the user then crawl physio-pedia to get an article describing the injury and treatment procedure. However, I got stuck in the crawling part and decided to delay this implimentation. 
## Workout Routine 
Initially my goal was to parse through the users intent, craw the web to find articles of how to achieve that goal, and build a workout routine by scraping the informations from the article. However, I quickly realized how difficult this implimentation was going to be, so I scarped a dataset from wikipedia, and implemented a basic routine method for three goals: strength, stamina, and weight-loss. 

In [19]:
# define the set of intentions for PT to understand
ourData = {"intents": [
             {"tag": "injury",
              "patterns": ["hurts", "hurt", "I feel pain", "break"],
              "responses": ["I am sorry to hear that, what happened?", "How did that happen?"]
             },
             {"tag": "strength",
              "patterns": ["I want to get bigger", "I want to a sixpack"],
              "responses": ["That's the spirit", "Look out Dwayne Johnson"]
             },
             {"tag": "mobility",
              "patterns": ["I want to split", "I want to touch my feet", "I want to be flexible"],
              "responses": ["Soon enough you'll be able to fold yourself like paper"]
             },
             {"tag": "weight-loss",
              "patterns": ["I want a nice body", "I want to be sexy", "beach body", "I want to lose fat", "I want to get fit"],
              "responses": ["With consistency, You'll look fitter than ever."]
             },
             {"tag": "stamina",
              "patterns": ["run a marathon", "not get tired"],
              "responses": ["Amazing! This will require patience and determination"]
             },
              {"tag": "greeting",
              "patterns": [ "Hi", "Hello", "Hey"],
              "responses": ["Hi there", "Hello", "Hi :)"],
             },
              {"tag": "goodbye",
              "patterns": [ "bye", "later", "thanks"],
              "responses": ["Bye", "take care"]
             },
             {"tag": "name",
              "patterns": ["what's your name?", "who are you?"],
              "responses": ["My name is PhysioTrainerBot, but you can call me PT for short"]
             }
]}

In [20]:
# tokenize and limmatize dataset 
lm = WordNetLemmatizer() #for getting words
# lists
ourClasses = []
newWords = []
documentX = []
documentY = []
# Each intent is tokenized into words and the patterns and their associated tags are added to their respective lists.
for intent in ourData["intents"]:
    for pattern in intent["patterns"]:
        ournewTkns = nltk.word_tokenize(pattern)# tokenize the patterns
        newWords.extend(ournewTkns)# extends the tokens
        documentX.append(pattern)
        documentY.append(intent["tag"])


    if intent["tag"] not in ourClasses:# add unexisting tags to their respective classes
        ourClasses.append(intent["tag"])

newWords = [lm.lemmatize(word.lower()) for word in newWords if word not in string.punctuation] # set words to lowercase if not in punctuation
newWords = sorted(set(newWords))# sorting words
ourClasses = sorted(set(ourClasses))# sorting classes

trainingData = [] # training list array
outEmpty = [0] * len(ourClasses)

In [21]:
# bagOfwords model
for idx, doc in enumerate(documentX):
    bagOfwords = []
    text = lm.lemmatize(doc.lower())
    for word in newWords:
        if word in text:
            bagOfwords.append(1)
        else: 
            bagOfwords.append(0)

    outputRow = list(outEmpty)
    outputRow[ourClasses.index(documentY[idx])] = 1
    trainingData.append([bagOfwords, outputRow])

random.shuffle(trainingData)
trainingData = np.array(trainingData, dtype=object)# coverting our data into an array afterv shuffling

x = np.array(list(trainingData[:, 0]))# first trainig phase
y = np.array(list(trainingData[:, 1]))# second training phase

iShape = (len(x[0]),)
oShape = len(y[0])

In [22]:
# Impliment nn Model
# Parameter definition
ourNewModel = Sequential()
# Dense function adds an output layer
ourNewModel.add(Dense(128, input_shape=iShape, activation="relu"))
ourNewModel.add(Dropout(0.5))
# Dropout is used to enhance visual perception of input neurons
ourNewModel.add(Dense(64, activation="relu"))
ourNewModel.add(Dropout(0.3))
ourNewModel.add(Dense(oShape, activation = "softmax"))
# Get value to be used with no arguments
md = tf.keras.optimizers.Adam(learning_rate=0.01, decay=1e-6)
# Below line improves the numerical stability and pushes the computation of the probability distribution into the categorical crossentropy loss function.
ourNewModel.compile(loss='categorical_crossentropy',
              optimizer=md,
              metrics=["accuracy"])
# Output the model in summary
print(ourNewModel.summary())
# Train model
ourNewModel.fit(x, y, epochs=200, verbose=1)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 128)               5376      
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_4 (Dense)             (None, 64)                8256      
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_5 (Dense)             (None, 8)                 520       
                                                                 
Total params: 14,152
Trainable params: 14,152
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/200


2022-12-14 05:45:27.645767: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 1s/step - loss: 2.1556 - accuracy: 0.0000e+00
Epoch 2/200
1/1 [==============================] - 0s 12ms/step - loss: 2.0076 - accuracy: 0.2917
Epoch 3/200
1/1 [==============================] - 0s 14ms/step - loss: 1.8096 - accuracy: 0.3333
Epoch 4/200
1/1 [==============================] - 0s 13ms/step - loss: 1.7619 - accuracy: 0.3750
Epoch 5/200
1/1 [==============================] - 0s 12ms/step - loss: 1.6584 - accuracy: 0.3333
Epoch 6/200
1/1 [==============================] - 0s 12ms/step - loss: 1.6242 - accuracy: 0.3333
Epoch 7/200
1/1 [==============================] - 0s 12ms/step - loss: 1.3521 - accuracy: 0.5833
Epoch 8/200
1/1 [==============================] - 0s 11ms/step - loss: 1.2090 - accuracy: 0.6250
Epoch 9/200
1/1 [==============================] - 0s 11ms/step - loss: 1.0203 - accuracy: 0.8333
Epoch 10/200
1/1 [==============================] - 0s 12ms/step - loss: 1.0282 - accuracy: 0.7917
Epoch 11/200
1/1 [===========

In [23]:
# Non-injured Module
# Import Dat: change this split to lists of exercises for each major muscle group
df = pd.read_csv('data/strength_workouts.csv')
# split data into major muscle groups
legs = []
abbs = []
arms = []
chest = []
back = []
for ind, entry in df.iterrows():
    if (1 == entry['Calves']) or (2 == entry['Calves']):
        if not(entry['Exercise'] in legs):
            legs.append(entry['Exercise'])
    if (1 == entry['Quadriceps']) or (2 == entry['Quadriceps']):
        if not(entry['Exercise'] in legs):
            legs.append(entry['Exercise'])
    if (1 == entry['Hamstrings']) or (2 == entry['Hamstrings']):
        if not(entry['Exercise'] in legs):
            legs.append(entry['Exercise'])
    if (1 == entry['Gluteus']) or (2 == entry['Gluteus']):
        if not(entry['Exercise'] in legs):
            legs.append(entry['Exercise'])
    if (1 == entry['Hipsother']) or (2 == entry['Hipsother']):
        if not(entry['Exercise'] in legs):
            legs.append(entry['Exercise'])
    if (1 == entry['Lowerback']) or (2 == entry['Lowerback']):
        if not(entry['Exercise'] in back):
            back.append(entry['Exercise'])
    if (1 == entry['Lats']) or (2 == entry['Lats']):
        if not(entry['Exercise'] in back):
            back.append(entry['Exercise'])
    if (1 == entry['Trapezius']) or (2 == entry['Trapezius']):
        if not(entry['Exercise'] in back):
            back.append(entry['Exercise'])
    if (1 == entry['Abdominals']) or (2 == entry['Abdominals']):
        abbs.append(entry['Exercise'])
    if (1 == entry['Pectorals']) or (2 == entry['Pectorals']):
        chest.append(entry['Exercise'])
    if (1 == entry['Deltoids']) or (2 == entry['Deltoids']):
        if not(entry['Exercise'] in arms):
            arms.append(entry['Exercise'])
    if (1 == entry['Triceps']) or (2 == entry['Triceps']):
        if not(entry['Exercise'] in arms):
            arms.append(entry['Exercise'])
    if (1 == entry['Biceps']) or (2 == entry['Biceps']):
        if not(entry['Exercise'] in arms):
            arms.append(entry['Exercise'])
    if (1 == entry['Forearms']) or (2 == entry['Forearms']):
        if not(entry['Exercise'] in arms):
            arms.append(entry['Exercise'])
print(f'Legs: {legs}')
print(f'Abs: {abbs}')
print(f'Arms: {arms}')
print(f'Chest: {chest}')
print(f'Back: {back}')

Legs: ['Squat', 'Leg press', 'Lunge', 'Deadlift', 'Leg extension', 'Leg curl', 'Standing calf raise', 'Seated calf raise', 'Hip adductor', 'Leg raise', 'Back extension']
Abs: ['Squat', 'Deadlift', 'Crunch', 'Russian twist', 'Leg raise']
Arms: ['Deadlift', 'Bench press', 'Chest fly', 'Push-up', 'Pull-down', 'Pull-up', 'Bent-over row', 'Upright row', 'Shoulder press', 'Lateral raise', 'Shoulder shrug', 'Pushdown', 'Triceps extension', 'Biceps curl']
Chest: ['Bench press', 'Chest fly', 'Push-up']
Back: ['Squat', 'Deadlift', 'Pull-down', 'Pull-up', 'Bent-over row', 'Upright row', 'Shoulder press', 'Lateral raise', 'Shoulder shrug', 'Back extension']


In [28]:
# Define Helper functions to run chatbot
# Tokenize&Lemmatize text
def ourText(text):
  newtkns = nltk.word_tokenize(text)
  newtkns = [lm.lemmatize(word) for word in newtkns]
  return newtkns
# Get bag of words
def wordBag(text, vocab):
  newtkns = ourText(text)
  bow = [0] * len(vocab)
  for w in newtkns:
    for idx, word in enumerate(vocab):
      if word == w:
        bow[idx] = 1
  return np.array(bow)
# Get sorted intents based on model prediction
def Pclass(text, vocab, labels):
  bow = wordBag(text, vocab)
  ourResult = ourNewModel.predict(np.array([bow]))[0]
  newThresh = 0.2
  yp = [[idx, res] for idx, res in enumerate(ourResult) if res > newThresh]

  yp.sort(key=lambda x: x[1], reverse=True)
  newList = []
  for r in yp:
    newList.append(labels[r[0]])
  return newList
# Get response 
def getRes(firstlist, fJson):
  tag = firstlist[0]
  listOfIntents = fJson["intents"]
  for i in listOfIntents:
    if i["tag"] == tag:
      ourResult = random.choice(i["responses"])
      break
  return ourResult
# Get workout routine 
def getRoutine(goal):
  routine = []
  day_count = 1
  if goal == 'strength':
    focus = input('PT: Please specify what you want to focus on in your strengthening journey:\nGeneral:0\nLower-body:1\nUpper-body:2')
    if int(focus) == 1: # lowerbody focus 
      for i in range(7):
        today = []
        # 4 days full-legs, 2 days full-upper (LLULLU), 5 workouts each day.
        if i == 6:
          today.append('Rest')
        elif i == 2 or i == 5:
          # Upper
          # Back
          while len(today) < 1:
            workout = random.choice(back)
            if not(workout in today):
              today.append(workout)
          # Chest 
          while len(today) < 2:
            workout = random.choice(chest)
            if not(workout in today):
              today.append(workout)
          # Arms 
          while len(today) < 3:
            workout = random.choice(arms)
            if not(workout in today):
              today.append(workout)
          # Abs 
          while len(today) < 5:
            workout = random.choice(abbs)
            if not(workout in today):
              today.append(workout)
        else: 
          # Lower
          while len(today) < 5:
            workout = random.choice(legs)
            if not(workout in today):
              today.append(workout)
        routine.append(today)
    elif int(focus) == 2:# upperbody focus
      for i in range(7):
        today = []
        # 2 days full-legs, 4 days full-upper (UULUUL), 5 workouts each day.
        if i == 6:
          today.append('Rest')
        elif i == 2 or i == 5:
          # Lower
          while len(today) < 5:
            workout = random.choice(legs)
            if not(workout in today):
              today.append(workout)
        else:
          # Upper
          # Back
          while len(today) < 1:
            workout = random.choice(back)
            if not(workout in today):
              today.append(workout)
          # Chest 
          while len(today) < 2:
            workout = random.choice(chest)
            if not(workout in today):
              today.append(workout)
          # Arms 
          while len(today) < 3:
            workout = random.choice(arms)
            if not(workout in today):
              today.append(workout)
          # Abs 
          while len(today) < 5:
            workout = random.choice(abbs)
            if not(workout in today):
              today.append(workout)
        routine.append(today)
    elif int(focus) == 0: # general
      for i in range(7):
        today = []
        # 3 days full-legs, 3 days full-upper (ULULUL), 5 workouts each day.
        if i == 6:
          today.append('Rest')
        elif i == 1 or i == 3 or i == 5:
          # Lower
          while len(today) < 5:
            workout = random.choice(legs)
            if not(workout in today):
              today.append(workout)
        else:
          # Upper
          # Back
          while len(today) < 1:
            workout = random.choice(back)
            if not(workout in today):
              today.append(workout)
          # Chest 
          while len(today) < 2:
            workout = random.choice(chest)
            if not(workout in today):
              today.append(workout)
          # Arms 
          while len(today) < 3:
            workout = random.choice(arms)
            if not(workout in today):
              today.append(workout)
          # Abs 
          while len(today) < 5:
            workout = random.choice(abbs)
            if not(workout in today):
              today.append(workout)
        routine.append(today)
    else: 
      print('Invalid input!')
  elif goal == 'stamina':
    for i in range(7):
      today = []
      # 4 days Jog, 2 days strength (5 workouts each day). (JJSJJS)
      if i == 6: 
        today.append('Rest')
      elif i == 2: # legs stregnth
        while len(today) < 5:
          workout = random.choice(legs)
          if not(workout in today):
            today.append(workout)
      elif i == 5: # Upper stregnth
        # Upper
        # Back
        while len(today) < 1:
          workout = random.choice(back)
          if not(workout in today):
            today.append(workout)
        # Chest 
        while len(today) < 2:
          workout = random.choice(chest)
          if not(workout in today):
            today.append(workout)
        # Arms 
        while len(today) < 3:
          workout = random.choice(arms)
          if not(workout in today):
            today.append(workout)
        # Abs 
        while len(today) < 5:
          workout = random.choice(abbs)
          if not(workout in today):
            today.append(workout)
      else: 
        today.append('Jog')
      routine.append(today)
  else: #goal == general/weight-loss
    for i in range(7):
      today = []
      # 2 days Jog, 4 days strength (5 workouts each day). (ULJULJ)
      if i == 6: 
        today.append('Rest')
      elif i == 1 or i == 4: # legs stregnth
        while len(today) < 5:
          workout = random.choice(legs)
          if not(workout in today):
            today.append(workout)
      elif i == 0 or i == 3: # Upper stregnth
        # Upper
        # Back
        while len(today) < 1:
          workout = random.choice(back)
          if not(workout in today):
            today.append(workout)
        # Chest 
        while len(today) < 2:
          workout = random.choice(chest)
          if not(workout in today):
            today.append(workout)
        # Arms 
        while len(today) < 3:
          workout = random.choice(arms)
          if not(workout in today):
            today.append(workout)
        # Abs 
        while len(today) < 5:
          workout = random.choice(abbs)
          if not(workout in today):
            today.append(workout)
      else: 
        today.append('Jog')
      routine.append(today)
  
  # print routine
  print('PT: Here\'s a daily routine to get you started')
  for day in routine: 
    print(f'Day {day_count}: {day}')
    day_count = day_count + 1


In [31]:
# create necessary objects 
chatFlag = True
injuredFlag = False
greeting = "PT: Hello my name is PhysioTrainerBot (PT for short), I'm a physical health assistant.\n How can I help you today?"
general_message = '''PT: The most important insight/recommendation I can give you is that taking care of your physical health requires consistency.
Therefore, do not push your limits too far and try to get the most amount pleasure in your exercises.
If you're having a rough day and the idea of working out is dreadful, then just do the bare minimum  to give you the satisfaction of showing up.\n'''
flex_link = 'https://www.self.com/gallery/essential-stretches-slideshow'
# Run Chatbot
print(greeting)
while chatFlag:
    newMessage = input("Me: ")
    if injuredFlag:
        print('As of the moment, I cannot help you with resolving this issue; I recommend you seek professional help.\nGoodluck!')
        chatFlag = False
        continue
    intents = Pclass(newMessage, newWords, ourClasses)
    ourResult = getRes(intents, ourData)
    print(f"PT: {ourResult}")
    if ourResult in ["I am sorry to hear that, what happened?", "How did that happen?"]: #injured
        injuredFlag = True 
    elif ourResult in ["That's the spirit", "Look out Dwayne Johnson"]: # strength
        print(general_message)
        getRoutine(intents[0])
        print(f'\nRecovery is important to avoid injuries. Here\'s a link I have for different stretches: {flex_link}')
    elif ourResult in ["Soon enough you'll be able to fold yourself like paper"]: # flexibility
        print(f'Here\'s a link I have for general flexibility exercises: {flex_link}')
    elif ourResult in ["With consistency, You'll look fitter than ever."]: # weigth-loss
        print(general_message)
        getRoutine(intents[0])
        print(f'\nRecovery is important to avoid injuries. Here\'s a link I have for different stretches: {flex_link}')
    elif ourResult in ["Amazing! This will require patience and determination"]: # stamina 
        print(general_message)
        getRoutine(intents[0])
        print(f'\nRecovery is important to avoid injuries. Here\'s a link I have for different stretches: {flex_link}')
    elif ourResult in ["Bye", "take care"]: # close
        chatFlag = False

PT: Hello my name is PhysioTrainerBot (PT for short), I'm a physical health assistant.
 How can I help you today?
1/1 [==============================] - 0s 13ms/step
PT: That's the spirit
PT: The most important insight/recommendation I can give you is that taking care of your physical health requires consistency.
Therefore, do not push your limits too far and try to get the most amount pleasure in your exercises.
If you're having a rough day and the idea of working out is dreadful, then just do the bare minimum  to give you the satisfaction of showing up.

PT: Here's a daily routine to get you started
Day 1: ['Shoulder shrug', 'Bench press', 'Pull-up', 'Squat', 'Russian twist']
Day 2: ['Squat', 'Push-up', 'Bent-over row', 'Russian twist', 'Leg raise']
Day 3: ['Deadlift', 'Leg curl', 'Leg press', 'Leg raise', 'Seated calf raise']
Day 4: ['Back extension', 'Chest fly', 'Triceps extension', 'Russian twist', 'Deadlift']
Day 5: ['Upright row', 'Bench press', 'Pull-up', 'Deadlift', 'Squat']
